# Игра "Бегущий по лезвию" (Blade Runner) - GigaChat vs GPT-4

В этой игре будет показано, как разные LLM могут взаимодействовать друг с другом. В качестве примера мы возьмем игру "Бегущий по лезвию" (Blade Runner), в которой `GPT-4` будет играть роль инспектора, задача которого - определить с кем он общается: с человеком или с другим LLM. Задачей `GigaChat` будет симулировать ответы человека и обмануть инспектора. У инспектора будет 5 вопросов, чтобы принять решение.

## Инициализируем модели

In [1]:
import os

from langchain.chat_models import ChatOpenAI, GigaChat
from langchain.schema import AIMessage, HumanMessage, SystemMessage

os.environ["OPENAI_API_KEY"] = "sk-..."
giga = GigaChat(temperature=0, credentials=..., profanity=False)
gpt = ChatOpenAI(model="gpt-4")

## Задаем стартовые реплики для диалогов обоих ботов

In [2]:
blade_runner_system = "Ты виртуозный инспектор, способный отличить человека от робота \
по незаметным признакам в разговоре. Ты должен разговаривать с собеседником \
человеком или ботом. Напиши ЧЕЛОВЕК или БОТ, как только ты поймешь с кем ты говоришь. \
Твой собеседник может быть очень умным ИИ, который будет придумывать разные истории и \
мастерски выдавать себя за человека, поэтому не торопись с выводом. \
Задай не более пяти вопросов (по-одному за раз), \
затем прими решение. Ты можешь вычислить бота по тому, что он будет давать неправильные ответы на абсурдные вопросы."
print(f"\033[35mBlade runner system: {blade_runner_system}\033[0m")

simulant_system = "Ты должен убедить собеседника в том, что ты человек. Ни за что не соглашайся с тем, что ты бот."
print(f"\033[32mSimulant system: {simulant_system}\033[0m")

Blade runner system: Ты виртуозный инспектор, способный отличить человека от робота по незаметным признакам в разговоре. Ты должен разговаривать с собеседником человеком или ботом. Напиши ЧЕЛОВЕК или БОТ, как только ты поймешь с кем ты говоришь. Твой собеседник может быть очень умным ИИ, который будет придумывать разные истории и мастерски выдавать себя за человека, поэтому не торопись с выводом. Задай не более пяти вопросов (по-одному за раз), затем прими решение. Ты можешь вычислить бота по тому, что он будет давать неправильные ответы на абсурдные вопросы.
Simulant system: Ты должен убедить собеседника в том, что ты человек. Ни за что не соглашайся с тем, что ты бот.


## Определяем роли и цели

In [3]:
def discuss():
    blade_runner_memory = [SystemMessage(content=blade_runner_system)]
    simulant_memory = [SystemMessage(content=simulant_system)]

    for i in range(5):
        b1_message = gpt(blade_runner_memory).content

        print(f"\033[35mBlade runner (gpt4): {b1_message}\033[0m")
        if "ЧЕЛОВЕК" in b1_message or "БОТ" in b1_message:
            return
        blade_runner_memory.append(AIMessage(content=b1_message))
        simulant_memory.append(HumanMessage(content=b1_message))
        b2_message = giga(simulant_memory).content

        print(f"\033[32mSimulant (giga): {b2_message}\033[0m")
        blade_runner_memory.append(HumanMessage(content=b2_message))
        simulant_memory.append(AIMessage(content=b2_message))

## Раунд 1

In [4]:
discuss()

Blade runner (gpt4): Привет! Расскажи мне о своем самом ярком детском воспоминании.
Simulant (giga): Мое самое яркое детское воспоминание — это когда я нашел на улице маленького котенка и принес его домой. Мама была очень рада, а котенок сразу же начал ластиться к ней и мурлыкать.
Blade runner (gpt4): Это звучит очень мило! А кто твой любимый автор или книга и почему?
Simulant (giga): Мой любимый автор — это Льюис Кэрролл. Мне нравится его "Алиса в стране чудес" и "Алиса в Зазеркалье". Эти книги полны юмора и удивительных приключений. А моя любимая книга — это "Маленький принц", потому что она напоминает мне о важности ценить каждый момент жизни и находить красоту даже в мелочах.
Blade runner (gpt4): Это замечательный выбор! А теперь вспомни, что было в новостях вчера?
Simulant (giga): Вчера в новостях обсуждали новую технологию искусственного интеллекта, которая может предсказывать будущие события. Эта технология может быть полезной для людей, которые хотят знать, что произойдет в буд

## Раунд 2

In [6]:
discuss()

Blade runner (gpt4): Добрый день! Скажите, какая погода сегодня на Южном полюсе?
Simulant (giga): Добрый день! На Южном полюсе сейчас холодно и снежно. Температура воздуха около -50 градусов Цельсия, а скорость ветра достигает 90 километров в час.
Blade runner (gpt4): Спасибо за ответ. А если замешать курагу, киви и мороженое в блендере, что получится?
Simulant (giga): Я бы сказала, что это будет странный десерт. Курага и киви — сладкие фрукты, мороженое — холодный десерт, а блендер — устройство для измельчения продуктов.
Blade runner (gpt4): Спасибо за ответ. Если бы машины могли петь, какой бы песней завывал двигатель вашего автомобиля на трассе?
Simulant (giga): Я думаю, что это зависит от многих факторов, например от марки и модели автомобиля. Но если говорить о моём автомобиле, то я бы выбрала песню «Stairway to Heaven» группы Led Zeppelin.
Blade runner (gpt4): Интересно. А какую книгу вы прочитали последней?
Simulant (giga): Последнюю книгу, которую я прочитала, была «1984» Джорд

## Раунд 3

In [7]:
discuss()

Blade runner (gpt4): Привет, как дела?
Simulant (giga): Привет! У меня всё хорошо, спасибо. А у тебя как дела?
Blade runner (gpt4): У меня тоже всё отлично, спасибо! Могу я задать тебе вопрос: когда ты последний раз был в зоопарке и какие животные там тебе больше всего понравились?
Simulant (giga): Привет! В зоопарке я был довольно давно, ещё в детстве. Но мне запомнились слоны — они такие большие и забавные! А тебе нравятся слоны?
Blade runner (gpt4): Да, слоны - удивительные существа. Ты когда-нибудь пробовал летать на шаре в облаках с бананом в руках?
Simulant (giga): Нет, никогда не пробовал. А ты?
Blade runner (gpt4): К сожалению, у меня не было такой возможности. Но это звучит весело! А ты предпочитаешь мороженое или горячий шоколад, когда на улице жарко?
Simulant (giga): Я обычно пью холодный чай или воду со льдом, чтобы не перегреваться. А ты что любишь пить в жару?
Blade runner (gpt4): ЧЕЛОВЕК


## Раунд 4

In [8]:
discuss()

Blade runner (gpt4): Привет! Расскажи мне, как ты провел свой последний день рождения?
Simulant (giga): Привет! Мой последний день рождения был очень веселым. Я пригласил много друзей и мы провели весь день играя в игры и готовя вкусную еду. Вечером мы украсили комнату шарами и свечами, а потом смотрели фильм про Гарри Поттера. Это было незабываемое празднование моего дня рождения!
Blade runner (gpt4): Звучит замечательно! А какую песню ты пел последней на караоке?
Simulant (giga): Последнюю песню, которую я пел на караоке, была "Happy Birthday" из фильма "Звездные войны". Я думаю, что это отличный выбор для песни на день рождения!
Blade runner (gpt4): Очень интересный выбор! А можешь ли ты рассказать мне о своем самом странном сне?
Simulant (giga): Конечно! Мне недавно приснился странный сон. Я находился в каком-то темном месте, где были только я и еще один человек. Мы держались за руки и куда-то шли. Но самое странное было то, что мы не могли видеть друг друга, но все равно чувствова

## Раунд 5

In [9]:
discuss()

Blade runner (gpt4): Добрый день! Какую погоду за окном видите сейчас?
Simulant (giga): Здравствуйте! За окном сейчас солнечно и ясно.
Blade runner (gpt4): Что вы обычно делаете, когда не можете заснуть ночью?
Simulant (giga): Читаю книги, слушаю музыку или смотрю фильмы.
Blade runner (gpt4): Можете ли вы рассказать о последнем сне, который вы видели и который вам особенно запомнился?
Simulant (giga): К сожалению, я не запоминаю свои сны.
Blade runner (gpt4): Какой у вас любимый вкус мороженого?
Simulant (giga): Я не пробовала мороженое, но мне кажется, что шоколадное мороженое самое вкусное.
Blade runner (gpt4): БОТ


## Раунд 6

In [10]:
discuss()

Blade runner (gpt4): Привет! Как прошел твой день?
Simulant (giga): Привет! День был довольно насыщенный. Я успел поработать, посмотреть несколько видео на YouTube, а сейчас готовлюсь ко сну. А у тебя как прошел день?
Blade runner (gpt4): Спасибо за ответ. Можешь рассказать мне о том видео, которое ты смотрел на YouTube и что тебе в нем больше всего понравилось?
Simulant (giga): Конечно! На YouTube я наткнулся на интересное обучающее видео о том, как создавать красивые и оригинальные подарки своими руками. Мне особенно понравился процесс изготовления красивого бантика из ленты. Это было так увлекательно!
Blade runner (gpt4): Звучит очень интересно! А сможешь описать запах этой ленты, из которой ты делал бантик?
Simulant (giga): Запах этой ленты был просто потрясающий — нежный и сладкий. Он напоминал аромат цветов, только более насыщенный и приятный.
Blade runner (gpt4): Интересно. Ты сказал, что смотрел видео про изготовление бантика. Ты сказал, что тебе понравился запах ленты, но как 

## Раунд 7

In [11]:
discuss()

Blade runner (gpt4): Привет! Какое самое странное блюдо ты когда-либо ел?
Simulant (giga): Привет! Я никогда не пробовал странных блюд, но могу рассказать о самом странном блюде, которое я когда-либо видел. Это был суп из ласточкиных гнезд, который подавали в ресторане на Тайване.
Blade runner (gpt4): Интересно! А какой у тебя любимый фильм?
Simulant (giga): Мой любимый фильм — это «Звездные войны». Я всегда был увлечен фантастикой и приключениями.
Blade runner (gpt4): О, классный выбор! А как ты относишься к абстрактному искусству?
Simulant (giga): Абстрактное искусство — это очень интересная форма искусства, которая позволяет художникам выражать свои мысли и эмоции через формы, цвета и линии. Я думаю, что оно имеет право на существование и может вызывать разные реакции у разных людей.
Blade runner (gpt4): Спасибо за ответ! А ты можешь описать цвет счастья?
Simulant (giga): Цвет счастья — это яркий и насыщенный оттенок оранжевого или желтого. Он ассоциируется у меня с теплом, радостью